In [1]:
import glob
import numpy as np
import cv2
import os
from config import EvalConfig as Config
from mvt.utils import draw_boxes, draw_box_ids

In [2]:
def load_detections(det_file):
    detections = []
    raw_data = np.genfromtxt(det_file, delimiter=',')
    for frame_idx in sorted(set(raw_data[:, 0])):
        idx = np.where(raw_data[:, 0] == frame_idx)
        detections.append(np.stack(raw_data[idx], axis=0)[:, 2:6])
    return detections

def load_groundtruth(gt_file, only_eval=False):
    """
    Args:
        gt_file (string): Full path of a MOT groundtruth txt file.
        
        only_eval (bool): If False load all groundtruth entries, otherwise
            load only entries in which column 7 is 1 indicating an entry that
            is to be considered during evaluation.
    """
    gt_boxes = []
    gt_ids = []
    gt_classes = []
    raw_data = np.genfromtxt(gt_file, delimiter=',')
    for frame_idx in sorted(set(raw_data[:, 0])):
        idx = np.where(raw_data[:, 0] == frame_idx)
        gt_box = np.stack(raw_data[idx], axis=0)[:, 2:6]
        gt_id = np.stack(raw_data[idx], axis=0)[:, 1]
        gt_class = np.stack(raw_data[idx], axis=0)[:, 7]
        consider_in_eval = np.stack(raw_data[idx], axis=0)[:, 6]
        consider_in_eval = consider_in_eval.astype(np.bool)
        if only_eval:
            gt_box = gt_box[consider_in_eval]
            gt_id = gt_id[consider_in_eval]
        gt_boxes.append(gt_box)
        gt_ids.append(gt_id)
        gt_classes.append(gt_class)
    return gt_ids, gt_boxes, gt_classes

In [3]:
seq_idx = "02"
video_file = os.path.join(Config.DATA_DIR, 'train', 'MOT17-{}-{}/seq.avi'.format(seq_idx, Config.DET_TYPE))
detections = load_detections(os.path.join(Config.DATA_DIR, 'train', 'MOT17-{}-{}/det/det.txt'.format(seq_idx, Config.DET_TYPE)))
gt_ids, gt_boxes, gt_classes = load_groundtruth(os.path.join(Config.DATA_DIR, 'train', 'MOT17-{}-{}/gt/gt.txt'.format(seq_idx, Config.DET_TYPE)), only_eval=True)

AttributeError: type object 'EvalConfig' has no attribute 'DET_TYPE'

In [15]:
set(list(np.hstack(gt_classes)))

{1.0, 2.0, 4.0, 7.0, 8.0, 9.0}

In [65]:
image_files = sorted(glob.glob(os.path.join(Config.DATA_DIR, 'train', 'MOT17-{}-{}/img1/*.jpg'.format(seq_idx, Config.DET_TYPE))))
for i, image_file in enumerate(image_files):
    image = cv2.imread(image_file, cv2.IMREAD_COLOR)

    image = draw_boxes(image, gt_boxes[i])
    image = draw_boxes(image, detections[i], color=(255,255,255))

    cv2.imshow("frame", image)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

array([[ 9.120e+02,  4.840e+02,  9.700e+01,  1.090e+02],
       [ 1.342e+03,  4.170e+02,  1.680e+02,  3.800e+02],
       [ 5.860e+02,  4.460e+02,  8.500e+01,  2.640e+02],
       [ 1.585e+03, -1.000e+00,  3.360e+02,  5.780e+02],
       [ 1.163e+03,  4.410e+02,  3.300e+01,  8.900e+01],
       [ 1.308e+03,  4.310e+02,  3.400e+01,  1.180e+02],
       [ 1.422e+03,  4.310e+02,  1.830e+02,  3.370e+02],
       [ 1.055e+03,  4.830e+02,  3.600e+01,  1.100e+02],
       [ 1.090e+03,  4.840e+02,  3.200e+01,  1.140e+02],
       [ 7.330e+02,  4.870e+02,  3.000e+01,  6.800e+01],
       [ 6.790e+02,  4.920e+02,  5.200e+01,  1.050e+02],
       [ 7.370e+02,  4.570e+02,  2.700e+01,  7.500e+01],
       [ 1.255e+03,  4.470e+02,  3.300e+01,  1.000e+02],
       [ 1.015e+03,  4.300e+02,  4.000e+01,  1.160e+02],
       [ 1.100e+03,  4.400e+02,  3.800e+01,  1.080e+02],
       [ 9.340e+02,  4.350e+02,  4.200e+01,  1.140e+02],
       [ 4.420e+02,  4.460e+02,  1.070e+02,  2.820e+02],
       [ 6.360e+02,  4.580e+02,